In [ ]:
import requests
import json

In [ ]:
connectionString=""
database="conta_digital"
collection="users"

In [ ]:
def get_collections(colletion: str):
  url = ""

  payload = json.dumps({
      "collection": colletion,
      "database": "conta_digital",
      "dataSource": "DadosPipeline",
      "projection": {
      }
  })
  headers = {
    'Content-Type': 'application/json',
    'Access-Control-Request-Headers': '*',
    'api-key': '',
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  resJson = json.loads(response.text)

  doc_list = resJson['documents']

  return doc_list

In [ ]:
users_df = spark.read.json(sc.parallelize(get_collections("users")))
application_df = spark.read.json(sc.parallelize(get_collections("applications")))
bank_accounts_df = spark.read.json(sc.parallelize(get_collections("bank_accounts")))
cards_df = spark.read.json(sc.parallelize(get_collections("cards")))
loans_df = spark.read.json(sc.parallelize(get_collections("loans")))
transactions_df = spark.read.json(sc.parallelize(get_collections("transactions")))

In [ ]:
# Conectando com Azure Data Lake usando SASToken
storageAccountName = "datalake3207a3fe3e4b6f13"
sasToken = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-06-14T09:10:54Z&st=2024-06-14T01:10:54Z&spr=https&sig=vaz4iKWnbTe0fpGwBNE%2BsuMSgCDo2amBYcu3w4IY5oA%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

In [ ]:
# Montando os containers
mount_adls('landing-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

In [ ]:
# Gravar dataframes em formato JSON na landaing zone
users_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/users")
transactions_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/transactions")
loans_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/loans")
cards_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/cards")
bank_accounts_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/bank_accounts")
application_df.write.format('json').save(f"/mnt/{storageAccountName}/landing-zone/application")